In [ ]:
import json
import pandas as pd

In [ ]:
#read json  files
receipts_df = pd.read_json('receipts.json.gz', compression='gzip', lines =True) # read line by line
users_df = pd.read_json('users.json.gz', compression='gzip', lines =True)
brands_df = pd.read_json('brands.json.gz', compression='gzip', lines =True)


In [ ]:
users_df.head()

In [ ]:
#filling missing values
users_df['lastLogin'] = users_df['lastLogin'].fillna({i: {} for i in users_df.index})

In [ ]:
# Two columns converting into DateTime
users_df['_id'] = pd.json_normalize(users_df['_id'])
users_df['createdDate'] = pd.to_datetime(pd.json_normalize(users_df['createdDate'])['$date'],unit = 'ms').astype('datetime64[s]')
users_df['lastLogin'] = pd.to_datetime(pd.json_normalize(users_df['lastLogin'])['$date'],unit = 'ms').astype('datetime64[s]')

In [ ]:
brands_df.head()

In [ ]:
#  convert dict column cpg: keys to column headers and values to rows
cpg_df = pd.json_normalize(brands_df['cpg']).add_prefix('cpg.')
#joining the two dfs
norm_brands_df = brands_df.join(cpg_df, how = 'outer')
#extracting id from dict
norm_brands_df['_id'] = pd.json_normalize(norm_brands_df['_id'])
#dropping cpg column 
norm_brands_df.drop(columns = 'cpg', axis = 1, inplace = True)
#Final dataset 
norm_brands_df.head()

In [ ]:
receipts_df.head()

In [ ]:
receipts_df.info()

In [ ]:
receipts_df = receipts_df.explode('rewardsReceiptItemList')
receipts_df.reset_index(inplace=True)

In [ ]:
#df each element into a row excluding 'ReceiptItemList' dict
receipts_df = receipts_df.explode('rewardsReceiptItemList')
receipts_df.reset_index(inplace=True)

In [ ]:
#  convert dict column rewardsReceiptItemList :keys to column headers and values to rows
receiptItemList_df = pd.json_normalize(receipts_df['rewardsReceiptItemList'], record_prefix = 'rewardsReceiptItemList_', errors = 'ignore').add_prefix('rewardsReceiptItemList_')


In [ ]:
#joining the two dfs
norm_receipts_df = receipts_df.join(receiptItemList_df, how = 'outer')
#dropping the 'rewardsReceiptItemList' coulmn 
norm_receipts_df.drop(columns = ['rewardsReceiptItemList'],axis = 1, inplace = True)
#final dataset
norm_receipts_df.head()

In [ ]:
#filling missing values
norm_receipts_df['finishedDate'] = norm_receipts_df['finishedDate'].fillna({i: {} for i in receipts_df.index})
norm_receipts_df['pointsAwardedDate'] = norm_receipts_df['pointsAwardedDate'].fillna({i: {} for i in receipts_df.index})
norm_receipts_df['purchaseDate'] = norm_receipts_df['purchaseDate'].fillna({i: {} for i in receipts_df.index})


In [ ]:
#extract data from dict and convert into DateTime
norm_receipts_df['createDate'] = pd.to_datetime(pd.json_normalize(norm_receipts_df['createDate'])['$date'],unit = 'ms').astype('datetime64[s]')
norm_receipts_df['dateScanned'] = pd.to_datetime(pd.json_normalize(norm_receipts_df['dateScanned'])['$date'],unit = 'ms').astype('datetime64[s]')
norm_receipts_df['finishedDate'] = pd.to_datetime(pd.json_normalize(norm_receipts_df['finishedDate'])['$date'],errors='coerce',unit = 'ms').astype('datetime64[s]')
norm_receipts_df['modifyDate'] = pd.to_datetime(pd.json_normalize(norm_receipts_df['modifyDate'])['$date'],unit = 'ms').astype('datetime64[s]')
norm_receipts_df['pointsAwardedDate'] = pd.to_datetime(pd.json_normalize(norm_receipts_df['pointsAwardedDate'])['$date'],unit = 'ms').astype('datetime64[s]')
norm_receipts_df['purchaseDate'] = pd.to_datetime(pd.json_normalize(norm_receipts_df['purchaseDate'])['$date'],unit = 'ms').astype('datetime64[s]')
